In [4]:
import tensorflow as ts
from tensorflow import keras
import numpy as np
import tensorflow.keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.callbacks import Callback

def custom_loss(y_true, y_pred,a, beta, e, x0, T):
    """
    Calculates the L2 norm of the difference between y_pred and a constant value, plus
    a scaled sum of y_pred, for each example in the batch. Returns the average L2 norm
    over all examples.

    Args:
    y_true: tensor of true labels
    y_pred: tensor of predicted labels

    Returns:
    A tensor representing the mean L2 norm over all examples.
    """
    
     # calculate the difference y_pred[i] - c * sum(y_pred)
    n = K.int_shape(y_pred)[1]
    hs = 1.0 / (n+1)
    diff = beta * e * (T + x0 - a) - y_pred - hs * beta * e * K.sum(y_pred, axis=1, keepdims=True)
    
    # calculate the L2 norm for each example
    norma = K.sqrt(K.sum(K.square(diff), axis=1))
    
    # return the average L2 norm over all training examples
    return K.mean(norma)



class SavePredictions(Callback):
    def __init__(self, x_val):
        self.x_val = x_val
        self.predictions = []
    
    def on_epoch_end(self, epoch, logs={}):
        # Faz as previsões no conjunto de validação
        y_pred = self.model.predict(self.x_val)
        
        # Salva as previsões em uma lista
        self.predictions.append(y_pred)
        
        # Salva as previsões em um arquivo a cada 5 épocas
        if epoch % 10 == 0:
            np.save('predictions_epoch{}.npy'.format(10000+epoch), y_pred)




In [8]:
# parâmetros
a, beta, e, x0, T = 0.0, 2.0, 1.0, 5.0, 1.0

# Tamanho da entrada, número de neurônios nas camadas ocultas
n, n1 = 100, 5


# Dados de treinamento
X_train = np.linspace(0,1,n).reshape(1,n)
y_train = np.linspace(0,1,n).reshape(1,n)

# Criação do modelo
model = Sequential()

# Adicionando as camadas ao modelo
model.add(InputLayer(input_shape=(n,)))
model.add(Dense(n1, activation='relu'))
model.add(Dense(n, activation='linear'))

# Compilando o modelo
loss = lambda y_true, y_pred: custom_loss(y_true, y_pred, a, beta, e, x0, T)
model.compile(loss=loss, optimizer='adam')


# Treinando o modelo
save_predictions = SavePredictions(X_train)
model.fit(X_train, y_train, epochs=1001, callbacks=[save_predictions])

# Fazendo uma previsão
y_pred = model.predict(X_train).ravel()

Epoch 1/1001
1/1 [==============================] - 1s 821ms/step - loss: 120.5831
Epoch 2/1001
1/1 [==============================] - 0s 97ms/step - loss: 120.4651
Epoch 3/1001
1/1 [==============================] - 0s 97ms/step - loss: 120.3567
Epoch 4/1001
1/1 [==============================] - 0s 76ms/step - loss: 120.2578
Epoch 5/1001
1/1 [==============================] - 0s 72ms/step - loss: 120.1681
Epoch 6/1001
1/1 [==============================] - 0s 77ms/step - loss: 120.0871
Epoch 7/1001
1/1 [==============================] - 1s 593ms/step - loss: 120.0134
Epoch 8/1001
1/1 [==============================] - 0s 82ms/step - loss: 119.9455
Epoch 9/1001
1/1 [==============================] - 0s 73ms/step - loss: 119.8820
Epoch 10/1001
1/1 [==============================] - 0s 83ms/step - loss: 119.8210
Epoch 11/1001
1/1 [==============================] - 0s 101ms/step - loss: 119.7607
Epoch 12/1001
1/1 [==============================] - 0s 83ms/step - loss: 119.7011
Epoch 13/1

In [7]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import imageio

# Define o número de iterações
directory = os.getcwd()
extensao = ".npy"
files = os.listdir()

filtered_files = sorted([file for file in files if file.endswith(extensao)])
num_iter = len(filtered_files)

# Loop pelas iterações
for i in range(num_iter):
    data = np.load(filtered_files[i])
    # Cria os dados do gráfico
    x = np.linspace(0, 1,n)
    y = data.ravel()
    
    # Cria a figura e o eixo
    fig, ax = plt.subplots()
    sns.lineplot(x=x, y=y, ax=ax)
    ax.set(xlim=[0,1],ylim=[-1,5])
    
    # Adiciona o número da iteração no canto superior direito
    ax.text(0.95, 0.95, f'Iteração: {i}', transform=ax.transAxes, ha='right', va='top')
    
    # Salva a figura em uma lista
    plt.savefig(f'iteracao_{i}.png', dpi=300)
    plt.close()

# Cria o gif a partir das imagens salvas
images = []
for i in range(num_iter):
    images.append(imageio.imread(f'iteracao_{i}.png'))
imageio.mimsave('animacao.gif', images, fps=5)

# Apaga os arquivos de imagem
for i in range(num_iter):
    os.remove(f'iteracao_{i}.png')

# Apaga os arquivos numpy
for file in filtered_files:
    os.unlink(file)


/var/folders/nx/r46v348x4k30hz1gmq94sv7m0000gp/T/ipykernel_18347/2184674127.py:36: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f'iteracao_{i}.png'))


In [11]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import io

# Define o número de iterações
directory = os.getcwd()
extensao = ".npy"
files = os.listdir()

filtered_files = sorted([file for file in files if file.endswith(extensao)])
num_iter = len(filtered_files)

# Cria uma lista para armazenar as imagens em memória
images = []

# Loop pelas iterações
for i in range(num_iter):
    data = np.load(filtered_files[i])
    # Cria os dados do gráfico
    x = np.linspace(0, 1,n)
    y = data.ravel()
    
    # Cria a figura e o eixo
    fig, ax = plt.subplots()
    sns.lineplot(x=x, y=y, ax=ax)
    ax.set(xlim=[0,1],ylim=[-1,5])
    
    # Adiciona o número da iteração no canto superior direito
    ax.text(0.95, 0.95, f'Iteração: {i}', transform=ax.transAxes, ha='right', va='top')
    
    # Salva a figura em memória como uma imagem
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png', dpi=300)
    buffer.seek(0)
    image = Image.open(buffer)
    images.append(image)
    
    # Fecha a figura
    plt.close()

# Salva o GIF a partir das imagens em memória
images[0].save('animacao.gif',
               save_all=True,
               append_images=images[1:],
               duration=100,
               loop=0)

# Apaga os arquivos numpy
# for file in filtered_files:
#     os.unlink(file)

In [14]:
type(filtered_files[0])

str